In [8]:
import deeplift
import sys, os
scriptsDir = os.environ.get("KERAS_DIR");                                       
if (scriptsDir is None):                                                        
    raise Exception("Please set environment variable KERAS_DIR to point to deeplearning repo");
sys.path.insert(0,scriptsDir);
import keras
from keras.models import model_from_yaml

model = model_from_yaml(open("modelsDir_runs/record_0_model_i4fMH_modelYaml.yaml").read())
model.load_weights("modelsDir_runs/record_0_model_i4fMH_modelWeights.h5")

In [9]:
scriptsDir = os.environ.get("UTIL_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable UTIL_SCRIPTS_DIR to point to the deeplift code");
sys.path.insert(0,scriptsDir);
from importDataPackage import importData
trainData, validData, testData = importData.loadTrainTestValidFromYaml("yaml/features.yaml",
                                                                       "yaml/labels.yaml",
                                                                       "yaml/splits.yaml")

(0, 'rows skipped from', 'features.gz')
Returning desired dict
Making numpy arrays out of the loaded files
('train', 'shapeX', (80000, 2))
('train', 'shapeY', (80000, 1))
('valid', 'shapeX', (10000, 2))
('valid', 'shapeY', (10000, 1))
('test', 'shapeX', (10000, 2))
('test', 'shapeY', (10000, 1))


In [10]:
model.layers[0].get_weights()

[array([[[ -1.10324550e+00],
         [  4.67225276e-02]],
 
        [[ -3.18271732e+00],
         [  2.00881809e-02]],
 
        [[  2.65948105e+00],
         [ -2.66333723e+00]],
 
        [[ -2.65702462e+00],
         [  2.67485046e+00]],
 
        [[  2.65231419e+00],
         [  2.66446018e+00]],
 
        [[  1.15762377e+00],
         [  1.29820837e-03]],
 
        [[ -2.83885859e-02],
         [ -2.68582249e+00]],
 
        [[  3.21576929e+00],
         [  1.74058396e-02]],
 
        [[ -2.66091681e+00],
         [ -2.66848087e+00]],
 
        [[  3.62684531e-03],
         [  2.71813583e+00]]], dtype=float32), array([[-0.04971155],
        [-2.32095981],
        [-3.23612142],
        [-3.2635138 ],
        [-3.25123096],
        [-0.08200027],
        [-1.51762187],
        [-2.37534022],
        [-3.25674224],
        [-1.56303227]], dtype=float32)]

In [11]:
import deeplift.blobs as blobs
input_layer = blobs.Input_FixedDefault(default=0.0, num_dims=None, shape=(None, 2))
dense_nodes = []
feature_weights = model.layers[0].get_weights()[0]
feature_biases = model.layers[0].get_weights()[1]
for i in range(len(feature_weights)):
    dense_nodes.append(blobs.Dense(W=feature_weights[i], b=feature_biases[i]))
    dense_nodes[-1].set_inputs(input_layer)
merge_layer = blobs.MaxMerge(axis=1, temp=1.0)
merge_layer.set_inputs(dense_nodes)
merge_layer.build_fwd_pass_vars()
merge_layer.set_scoring_mode(blobs.ScoringMode.OneAndZeros)
merge_layer.set_active()
input_layer.update_mxts()

In [12]:
import theano
keras_fprop = theano.function([model.layers[0].input], model.layers[-1].get_output(train=False), allow_input_downcast=True)

In [13]:
deeplift_fprop = theano.function([input_layer.get_activation_vars()], merge_layer.get_activation_vars(), allow_input_downcast=True)

In [21]:
deeplift_fprop2 = theano.function([input_layer2.get_activation_vars()], maxout_layer.get_activation_vars(), allow_input_downcast=True)

In [22]:
import numpy as np
print(np.max(np.abs(np.array(keras_fprop(validData.X))-np.array(deeplift_fprop(validData.X)))))
print(np.max(np.abs(np.array(keras_fprop(validData.X))-np.array(deeplift_fprop2(validData.X)))))

9.53674e-07
0.0


In [23]:
deeplift_contribs_func = theano.function([input_layer.get_activation_vars()], input_layer.get_target_contrib_vars(), allow_input_downcast=True)

In [16]:
grad_times_inp_func = theano.function([model.layers[0].input], theano.grad(theano.tensor.sum(model.layers[-1].get_output(train=False)[:,0]),
                                                                      model.layers[0].input)*model.layers[0].input, allow_input_downcast=True)

In [24]:
true_contribs = np.square(validData.X)
deeplift_contribs = deeplift_contribs_func(validData.X)
grad_times_inp = grad_times_inp_func(validData.X)

In [28]:
deeplift_contribs_func2 = theano.function([input_layer2.get_activation_vars()], input_layer2.get_mxts(), allow_input_downcast=True)
deeplift_contribs2 = deeplift_contribs_func2(validData.X)
print(deeplift_contribs2)

[[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 ..., 
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]]


In [37]:
import deeplift.blobs as blobs
reload(blobs.maxout)
reload(blobs)
input_layer2 = blobs.Input_FixedDefault(default=0.0, num_dims=None, shape=(None, 2))
maxout_layer = blobs.Maxout(W=model.layers[0].get_weights()[0], b=model.layers[0].get_weights()[1])
maxout_layer.set_inputs(input_layer2)
maxout_layer.build_fwd_pass_vars()
maxout_layer.set_scoring_mode(blobs.ScoringMode.OneAndZeros)
maxout_layer.set_active()
input_layer2.update_mxts()

debug_func = theano.function([input_layer2.get_activation_vars()], maxout_layer._debug_negative_change_vec_mask, allow_input_downcast=True)
debug_vals = debug_func(validData.X)
print(debug_vals)

[[[[0]
   [0]
   [1]
   ..., 
   [1]
   [1]
   [0]]

  [[1]
   [0]
   [1]
   ..., 
   [1]
   [1]
   [1]]

  [[0]
   [0]
   [0]
   ..., 
   [0]
   [0]
   [0]]

  ..., 
  [[0]
   [0]
   [1]
   ..., 
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [1]
   ..., 
   [1]
   [0]
   [0]]

  [[1]
   [0]
   [1]
   ..., 
   [1]
   [1]
   [0]]]


 [[[0]
   [0]
   [0]
   ..., 
   [1]
   [0]
   [1]]

  [[1]
   [0]
   [1]
   ..., 
   [1]
   [0]
   [1]]

  [[1]
   [0]
   [0]
   ..., 
   [1]
   [0]
   [1]]

  ..., 
  [[0]
   [0]
   [0]
   ..., 
   [0]
   [0]
   [1]]

  [[1]
   [1]
   [1]
   ..., 
   [1]
   [0]
   [1]]

  [[0]
   [0]
   [0]
   ..., 
   [0]
   [0]
   [0]]]


 [[[0]
   [1]
   [0]
   ..., 
   [0]
   [1]
   [0]]

  [[0]
   [0]
   [0]
   ..., 
   [0]
   [0]
   [0]]

  [[1]
   [1]
   [0]
   ..., 
   [1]
   [1]
   [1]]

  ..., 
  [[1]
   [1]
   [0]
   ..., 
   [0]
   [1]
   [1]]

  [[0]
   [1]
   [0]
   ..., 
   [0]
   [0]
   [0]]

  [[1]
   [1]
   [0]
   ..., 
   [0]
   [1]
   [0]]]


 ..., 
 [[[0]
  

In [25]:
print(np.max(np.square(true_contribs-deeplift_contribs)))
print(np.max(np.square(true_contribs-deeplift_contribs2)))
print(np.max(np.square(true_contribs-grad_times_inp)))

0.350772899701
nan
6.6837174972
